# Sprawozdanie z **Obliczeń ewolucyjnych**
## Zadanie 1 - Algorytm *Particle Swarm Optimization*

# Configuration
Przy pomocy klasy Config można wybrać zastosowany algorytm oraz określić jego parametry:    
- w - współczynnik zmęczenia.   
- c1 - współczynnik przyspieszenia w kierunku własnego minimum cząstki
- c2 - współczynnik przyspieszenia w kierunku globalnego minimum cząstek
- iterations - ilość iteracji algorytmu  
- target_error - najniższy stopień błędu - po osiągnięciu tego wyniku algorytm zatrzymuje się  
- n_particles - ilość cząstek  
- arguments_dimensions - liczba wymiarów, w których rozkładane są cząsteczki   
- d_min - minimalna wartość współrzędnej dla każdego z wymiarów   
- d_max - maksymalna wartość współrzędnej dla każdego z wymiarów   
- fitness - wybór funkcji dostosowującej położenie cząstek         
 Dla celów badawczych, w dalszej części zaprezentowane są różne zestawy wartości konfiguracyjnych.

In [166]:
!pip install deap k3d > /dev/null
from pathlib import Path
import json

class Config(object):
    def __init__(self, algorithmType, w, c1, c2, custom_c2, iterations, target_error, n_particles, arguments_dimensions, d_min, d_max, fitness):
        self.algorithmType = algorithmType
        self.w = w
        self.c1 = c1
        self.c2 = c2
        self.custom_c2 = custom_c2
        self.iterations = iterations
        self.target_error = target_error
        self.n_particles = n_particles
        self.arguments_dimensions = arguments_dimensions
        self.d_min = d_min
        self.d_max = d_max
        self.d_max = d_max
        self.fitness = fitness

def as_config(dct):
    return Config(
        dct['algorithmType'],
        dct['w'],
        dct['c1'],
        dct['c2'],
        dct['custom_c2'] if dct.get('custom_c2') else False,
        dct['iterations'], 
        dct['target_error'], 
        dct['n_particles'],
        dct['arguments_dimensions'],
        dct['d_min'], 
        dct['d_max'],
        dct['fitness']
        )

# Genetic algorithm using deap
W celu porównania samodzielnie zaimplementowanego algorytmu PSO do algorytmów genetycznych, poniżej zaimplementowano funkcję zwracające wyniki działania tego algorytmu z wykorzystaniem biblioteki "deap".  

In [168]:
import random
import numpy
import inspect
from deap import algorithms, base, creator, tools

# translates into tuple
def gaFitness(individual):
    return fitness(individual),

def cxTwoPointCopy(ind1, ind2):
    size = len(ind1)
    cxpoint1 = random.randint(1, size)
    cxpoint2 = random.randint(1, size - 1)
    if cxpoint2 >= cxpoint1:
        cxpoint2 += 1
    else: # Swap the two cx points
        cxpoint1, cxpoint2 = cxpoint2, cxpoint1

    ind1[cxpoint1:cxpoint2], ind2[cxpoint1:cxpoint2] \
        = ind2[cxpoint1:cxpoint2].copy(), ind1[cxpoint1:cxpoint2].copy()
        
    return ind1, ind2

def genetic():
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", numpy.ndarray, fitness=creator.FitnessMin)

    toolbox = base.Toolbox()
    toolbox.register("attr_bool", lambda: (cfg.d_max-cfg.d_min)*random.random() + cfg.d_min)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=cfg.arguments_dimensions)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("evaluate", gaFitness)
    toolbox.register("mate", cxTwoPointCopy)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)
    pop = toolbox.population(n=cfg.n_particles)
    hof = tools.HallOfFame(1, similar=numpy.array_equal)
    
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)
    
    algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=cfg.iterations, stats=stats, halloffame=hof)

    return pop, stats, hof

# Particle swarm algorithm classes
Poniżej przedstawione są klasy repezentujące "rój"
### Particle
Klasa reprezentacja jedną cząsteczkę w przestrzeni poszukiwań:  
- position - pozycja w przestrzeni zdefiniowanej w pliku konfiguracyjnym. 
- pbest_position - najlepsza pozycja cząsteczki
- pbest_value - najlepsza osiągnięta przez tę czastęczkę wartość
- velocity - wektor, który przechowuje informacje nt. tzw prędkości cząsteczki.  
### Space
Klasa reprezentująca przestrzeń poszukiwań, w której poruszają się cząsteczki
- particles[] - tablica cząsteczek
- gbest_position - globalnie najlepsza pozycja cząsteczki - inicjowana losowo 
- gbest_value - najlepsza wartość funkcji przystosowania dowolnej z cząsteczek

In [169]:
import random
import numpy as np

class Particle():
    def __init__(self):
        self.position = (cfg.d_max - cfg.d_min) * np.random.random(cfg.arguments_dimensions) + cfg.d_min
        self.pbest_position = self.position
        self.pbest_value = float('inf')
        self.velocity = np.zeros(cfg.arguments_dimensions)
    
    def move(self):
        self.position = self.position + self.velocity
        for i in range(len(self.position)):
            if self.position[i] > cfg.d_max:
                self.position[i] = cfg.d_max
            if self.position[i] < cfg.d_min:
                self.position[i] = cfg.d_min

    def __str__(self):
        print(f'My position is {self.position}, my pbest is {self.pbest_position}')

class Space():
    def __init__(self):
        self.particles = []
        self.gbest_value = float('inf')
        self.gbest_position = np.array([x * np.random.random_sample()*cfg.d_max for x in range(cfg.arguments_dimensions)])

    def print_particles(self):
        for particle in self.particles:
            particle.__str__()

    def set_pbest(self):
        for particle in self.particles:
            fitness_cadidate:float = fitness(particle.position)
            if(particle.pbest_value > fitness_cadidate):
                particle.pbest_value = fitness_cadidate
                particle.pbest_position = particle.position

    def set_gbest(self):
        for particle in self.particles:
            best_fitness_cadidate = fitness(particle.position)
            if(self.gbest_value > best_fitness_cadidate):
                self.gbest_value = best_fitness_cadidate
                self.gbest_position = particle.position

    def move_particles(self):
        for particle in self.particles:
            new_velocity = (cfg.w*particle.velocity) + (cfg.c1*np.random.random_sample()) * (particle.pbest_position - particle.position) + \
                            (np.random.random_sample()*cfg.c2) * (self.gbest_position - particle.position)
            particle.velocity = new_velocity
            particle.move()


# Plots
Wykorzystano bibliotekę k3d w celu przedstawienia trójwymiarowych wykresów "roju", tam gdzie było to możliwe.

In [170]:
from numpy import exp,array,float32
import k3d
from uuid import uuid4

def plot_particles(particles):
    if len(particles) == 0:
        print("No particles to visualize !")
        return
    if len(particles[0].position) != 2:
        print("I can only visualize three dimensional function !")
        return
    x = list(map(lambda p: (p.position[0], p.position[1], fitness(p.position)), particles))
    
    plot = k3d.plot(name=str(uuid4()))
    plt_points = k3d.points(positions=x, point_size=0.2)
    plot += plt_points
    plt_points.shader='3d'
    plot.display()

# Fitness functions
Poniższa sekcja przestawia funkcje dostosowania cząsteczek:  
Sphere
![title](pics/sphere.png)
F2
![title](pics/f2.png)
Griewank
![title](pics/griewank.png)   
Rosenbrock
![title](pics/rosen.png)   
Schwefel
![title](pics/schwefel.png)   
Zakharov
![title](pics/zakharov.png)   

In [193]:
from numpy import sum, asarray_chkfinite, arange, prod, sqrt, cos, sin
from functools import reduce
def fitness(position, fr=4000):
    if cfg.fitness == "sincos":
        return sin(position[0]) + cos(position[1])
    if cfg.fitness == "sincos2":
        return sin(position[0]) + cos(position[1]) + position[0] + position[1]
    elif cfg.fitness == "sphere":
        return reduce(lambda p,n: p + n**2, position)
    elif cfg.fitness == "rosen":
        return sum(100.0*(position[1:]-position[:-1]**2.0)**2.0 + (position[:-1]-1)**2.0)
    elif cfg.fitness == "griewank":
        n = len(position)
        j = arange( 1., n+1 )
        s = sum( position**2 )
        p = prod( cos( position / sqrt(j) ))
        return s/fr - p + 1
    elif cfg.fitness == "zakharov":
        x = np.asarray_chkfinite(position)
        n = len(x)
        j = np.arange( 1., n+1 )
        s2 = sum( j * x ) / 2
        return sum( x**2 ) + s2**2 + s2**4
    elif cfg.fitness == "f2":
        s = 0
        for x in range(1, len(position)):
            s = s + (position[x]-x)**2
        return s
    elif cfg.fitness == "serduszko":
        x, y = position[0], position[1]
        return -80/(-80*x**2 - 9*y**2) + (-41472000*x**6 - 102643200*x**4*y**2 + 41472000*x**4 - 21520080*x**2*y**4 + 9331200*x**2*y**2 + sqrt((-41472000*x**6 - 102643200*x**4*y**2 + 41472000*x**4 - 21520080*x**2*y**4 + 9331200*x**2*y**2 - 1180980*y**6 + 524880*y**4 - 27648000)**2 - 764411904000000) - 1180980*y**6 + 524880*y**4 - 27648000)**(1/3)/(3*2**(1/3)*(-80*x**2 - 9*y**2)) + (19200*2**(1/3)/((-80*x**2 - 9*y**2)*(-41472000*x**6 - 102643200*x**4*y**2 + 41472000*x**4 - 21520080*x**2*y**4 + 9331200*x**2*y**2 + sqrt((-41472000*x**6 - 102643200*x**4*y**2 + 41472000*x**4 - 21520080*x**2*y**4 + 9331200*x**2*y**2 - 1180980*y**6 + 524880*y**4 - 27648000)**2 - 764411904000000) - 1180980*y**6 + 524880*y**4 - 27648000)**(1/3)))

# App
Pętla programowa odpowiedzialna za wywoływanie logiki wybranego algorytmu PSO lub GA.

In [172]:
import numpy as np
class App():
    def run(self):
        if(cfg.algorithmType == "pso"):
            c2_begin = cfg.c2
            search_space = Space()
            particles_vector = [Particle() for _ in range(cfg.n_particles)]
            search_space.particles = particles_vector
            plot_particles(search_space.particles)            
#             search_space.print_particles()
            iteration = 0
            while iteration < cfg.iterations:
                if cfg.custom_c2 == True:
                    search_space.c2 = c2_begin * ((cfg.iterations - iteration)/cfg.iterations)
                last_gbest = search_space.gbest_value
                search_space.set_pbest()
                search_space.set_gbest()
                search_space.move_particles()
                iteration += 1
                if abs(last_gbest - search_space.gbest_value) < cfg.target_error:
                    break
            print("The best solution is: ", np.append(search_space.gbest_position, fitness(search_space.gbest_position)), " in n_iterations: ", iteration)
            plot_particles(search_space.particles)
#             search_space.print_particles()
        elif(cfg.algorithmType == "ga"):
            genetic()

        else:
            print(f"Unknown algorithm type: {cfg.algorithmType}")

### Set seeds

In [187]:
from numpy import random as nrand
nrand.seed(0)
from random import seed
seed(0)

# Badania
Przeprowadzono eksperyment uruchomień algorytmu dla różnych zestawów parametrów konfiguracyjnych. Poniżej przedstawiono poszczególne przykłady wraz z wynikami w postaci wykresów 3D lub najlepszego osiągniętego wyniku.

### Funkcja *Sphere*

In [194]:
json_config ="""
{
    "algorithmType": "pso",
    "fitness": "serduszko",
    "w": 0.5,
    "c1": 0.8,
    "c2": 0.9,
    "target_error": 1e-8,
    "iterations": 300,
    "n_particles": 100,
    "arguments_dimensions": 2,
    "d_min": -10,
    "d_max": 10
}"""
cfg = json.loads(json_config, object_hook = as_config)
App().run()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in double_scalars
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in sqrt


Output()

The best solution is:  [ 0.0000  4.7247     nan]  in n_iterations:  300


Output()

In [174]:
json_config ="""
{
    "algorithmType": "pso",
    "fitness": "sincos",
    "w": 0.6,
    "c1": 0.2,
    "c2": 0.4,
    "custom_c2": true,
    "target_error": 0,
    "iterations": 400,
    "n_particles": 1000,
    "arguments_dimensions": 2,
    "d_min": -10,
    "d_max": 10
}"""
cfg = json.loads(json_config, object_hook = as_config)
App().run()

Output()

The best solution is:  [-1.5708  3.1416 -2.0000]  in n_iterations:  400


Output()

In [188]:
json_config ="""
{
    "algorithmType": "pso",
    "fitness": "zakharov",
    "w": 0.5,
    "c1": 0.7,
    "c2": 0.8,
    "custom_c2": false,
    "target_error": 0.001,
    "iterations": 200,
    "n_particles": 1600,
    "arguments_dimensions": 20,
    "d_min": -10,
    "d_max": 10
}"""
cfg = json.loads(json_config, object_hook = as_config)
App().run()

I can only visualize three dimensional function !
The best solution is:  [ 0.1668  0.2322 -0.4056 -0.0048  0.4179 -0.7187  0.1097  0.0626  0.0667
  0.1152 -0.2649  0.3112 -0.0218  0.2691 -0.5545  0.1036 -1.1605 -0.3083
  0.6554  0.7634  4.0032]  in n_iterations:  69
I can only visualize three dimensional function !
